# Real-World Document Embedding for Fast Search

Vector databases have gotten much more popular for querying text data. But how do we get a quick and dirty prototype started? 
<br><br>In this demo, we'll pull data from an internal source like you would at your workplace/larger dev project and run vector search inside this notebook.


#### Goal: Take real-world document data and make fast queries with Vector Search

## Contents
- Baseline Architecture
    - Vector DB common questions
- Implementation
    - Data
    - Code


Our workflow will be as follows: MongoDB data -> OpenAI embedding -> Search with Milvus

### Baseline Architecture

- Data Source: MongoDB
- Embeddings Generator: OpenAI Embeddings API
- Vector DB/Vector Search: Milvus Lite

![VDB Basic Architecture](./vdb.png)

#### Questions we need to answer:
- What's the point of this?
- How is this different from how we have been doing search?
- Do I need to store all of my data in a vector database?
- Does this save time, money, effort?

The key idea is that we can run search based on vector embeddings instead of the data itself. 

Traditional search systems, and even modern search systems, rely on processing parts of the dataset during inference. 

No, we just need to store the vector embeddings. The fastest way is to store a pointer/index of the original data along with its embedding.

Vector search using approximate nearest neighbors (ANN) has a much lower time complexity, resulting in __orders of magnitude__ less compute needed.

## Implementation

### Dataset: TechCrunch RSS News Feed

We have a prepared dataset of news from TechCrunch's RSS Feed, and we load it into a Pandas dataframe for readability. **fulltext** is the cleaned text data from each news article.

#### Let's pull our document data from MongoDB

In [1]:
from pymongo.mongo_client import MongoClient
from tqdm import tqdm
import pandas as pd
import openai
import pymilvus
import milvus
import os

In [2]:
#Get posts
mc = MongoClient(os.getenv('MONGO_URI'))
posts = pd.json_normalize(list(mc['data']['rss_data'].find({}))).rename(columns={'_id': 'title'})
#How the data is structured
posts.head()

,title,link,published,description,fulltext
0,AI leaders warn Senate of twin risks: Moving t...,https://techcrunch.com/2023/07/25/ai-leaders-w...,"Tue, 25 Jul 2023 23:23:15 +0000",Leaders from the AI research world appeared be...,Leaders from the AI research world appeared be...
1,"After federal court win, Ripple’s legal head s...",https://techcrunch.com/2023/07/25/ripple-sec-r...,"Tue, 25 Jul 2023 16:25:14 +0000","While Stu Alderoty is Ripple’s lawyer, his adv...",\nIt’s been almost two weeks since a federal c...
2,Akhetonics raises capital for its all-optical ...,https://techcrunch.com/2023/07/25/akhetonics-r...,"Tue, 25 Jul 2023 19:06:15 +0000",Traditional processors from the likes of Intel...,Traditional processors from the likes of Intel...
3,Amazon will now warn consumers of recalls and ...,https://techcrunch.com/2023/07/25/amazon-will-...,"Tue, 25 Jul 2023 20:40:02 +0000",Amazon today introduced a new feature that wil...,Amazon today introduced a new feature that wil...
4,Apple’s app tracking triggers statement of obj...,https://techcrunch.com/2023/07/25/apple-att-an...,"Tue, 25 Jul 2023 17:04:21 +0000","Bad news could be brewing for Apple in France,...","Bad news could be brewing for Apple in France,..."


#### Generate Embeddings

We use OpenAI's embeddings, but in practice you can use any text embedding model. Make sure to use TQDM's progress bar (so you know how long each iteration takes)!

In [3]:
openai.api_key = os.getenv("OPENAI_KEY")
model_id = "text-embedding-ada-002"

def get_embedding(text: str):
    return openai.Embedding.create(input=text, model=model_id)['data'][0]['embedding']

embeddings = []
for idx, row in tqdm(posts.iterrows()):
    embedding = get_embedding(row['fulltext'])
    embeddings.append(embedding)

20it [00:04,  4.66it/s]


In [4]:
#Add embeddings to our dataframe
posts['embeddings'] = embeddings
posts.head()

,title,link,published,description,fulltext,embeddings
0,AI leaders warn Senate of twin risks: Moving t...,https://techcrunch.com/2023/07/25/ai-leaders-w...,"Tue, 25 Jul 2023 23:23:15 +0000",Leaders from the AI research world appeared be...,Leaders from the AI research world appeared be...,"[0.013138867914676666, -0.01878361403942108, 0..."
1,"After federal court win, Ripple’s legal head s...",https://techcrunch.com/2023/07/25/ripple-sec-r...,"Tue, 25 Jul 2023 16:25:14 +0000","While Stu Alderoty is Ripple’s lawyer, his adv...",\nIt’s been almost two weeks since a federal c...,"[0.01812366023659706, -0.010685862973332405, -..."
2,Akhetonics raises capital for its all-optical ...,https://techcrunch.com/2023/07/25/akhetonics-r...,"Tue, 25 Jul 2023 19:06:15 +0000",Traditional processors from the likes of Intel...,Traditional processors from the likes of Intel...,"[-0.006664058659225702, -0.0057647209614515305..."
3,Amazon will now warn consumers of recalls and ...,https://techcrunch.com/2023/07/25/amazon-will-...,"Tue, 25 Jul 2023 20:40:02 +0000",Amazon today introduced a new feature that wil...,Amazon today introduced a new feature that wil...,"[-0.00531191099435091, -0.002216337714344263, ..."
4,Apple’s app tracking triggers statement of obj...,https://techcrunch.com/2023/07/25/apple-att-an...,"Tue, 25 Jul 2023 17:04:21 +0000","Bad news could be brewing for Apple in France,...","Bad news could be brewing for Apple in France,...","[-0.009665623307228088, 0.00716297747567296, 0..."


#### VectorDB initialization

In [7]:
#Refresh the Milvus instance if one already exists
# if default_server:
#     default_server.stop()
#     default_server.cleanup()
#     connections.disconnect('default')
#     del default_server
from milvus import default_server
from pymilvus import connections, utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
from datetime import datetime

COLLECTION_NAME = "RSS_POST_SEARCH"
#Depending on your embedding size, you should change the dimension.
#The quickest way I found was to try adding embeddings to the Collection object first,
#and the resulting bug will tell you which dimension your Milvus instance should accept.
DIMENSION = 12288
BATCH_SIZE = 64
TOPK = 3

#Time how long it takes to start Milvus instance
start = datetime.now()

default_server.start()
connections.connect(host="127.0.0.1", port=default_server.listen_port)
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)
    
    
# object should be inserted in the format of (title, date, location, speech embedding)
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=500),
    FieldSchema(name="link", dtype=DataType.VARCHAR, max_length=200),
    FieldSchema(name="date", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]
schema = CollectionSchema(fields=fields)
collection = Collection(name=COLLECTION_NAME, schema=schema)


#End time
total = datetime.now() - start
print("Milvus instance started in " + str(total) + " seconds")



    __  _________ _   ____  ______
   /  |/  /  _/ /| | / / / / / __/
  / /|_/ // // /_| |/ / /_/ /\ \
 /_/  /_/___/____/___/\____/___/ {Lite}

 Welcome to use Milvus!

 Version:   v2.2.12-lite
 Process:   5505
 Started:   2023-08-12 18:47:13
 Config:    /home/kkang2097/.milvus.io/milvus-server/2.2.12/configs/milvus.yaml
 Logs:      /home/kkang2097/.milvus.io/milvus-server/2.2.12/logs

 Ctrl+C to exit ...
Milvus instance started in 0:00:07.400267 seconds


In [8]:
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": DIMENSION},
}

collection.create_index(field_name="embedding", index_params=index_params)
collection.load()

In [9]:
#Add DataFrame data to Milvus
for idx, row in posts.iterrows():
    data = [[row['title']], [row['link']], [str(row['published'])], [row['embeddings']]]
    
    collection.insert(data)
collection.flush()

RPC error: [batch_insert], <ParamError: (code=1, message=Collection field dim is 12288, but entities field dim is 1536)>, <Time:{'RPC start': '2023-08-12 18:47:16.756462', 'RPC error': '2023-08-12 18:47:16.756503'}>


ParamError: <ParamError: (code=1, message=Collection field dim is 12288, but entities field dim is 1536)>

In [10]:
import time

#Embed our search query
query_string = 'Generative AI'
search_vec = [get_embedding('Generative AI')]


#Find relevant articles
start = time.time()
res = collection.search(
    data=search_vec,  # Embedded search value
    anns_field="embedding",  # Search across embeddings
    param={"metric_type": "L2",
            "params": {"nprobe": 10}},
    limit = TOPK,  # Limit to top_k results per search
    output_fields=["title"]  # Include title field in result
)
end = time.time()

#Show results
for hits_i, hits in enumerate(res):
    print("Title:", query_string)
    print("Search Time:", end-start)
    print("Results:")
    for hit in hits:
        print( hit.entity.get("title"), "----", hit.distance)
    print()

RPC error: [search], <MilvusException: (code=1, message=fail to search on all shard leaders, err=All attempts results:
attempt #1:code: UnexpectedError, error: fail to Search, QueryNode ID=2, reason=stream operation failed: [UnexpectedError] Assert "field_meta.get_sizeof() == element.line_sizeof_" at /__w/milvus-lite/milvus-lite/milvus_binary/milvus/internal/core/src/query/Plan.cpp:48
 => vector dimension mismatch
attempt #2:context canceled
)>, <Time:{'RPC start': '2023-08-12 18:47:17.419110', 'RPC error': '2023-08-12 18:47:17.422768'}>


MilvusException: <MilvusException: (code=1, message=fail to search on all shard leaders, err=All attempts results:
attempt #1:code: UnexpectedError, error: fail to Search, QueryNode ID=2, reason=stream operation failed: [UnexpectedError] Assert "field_meta.get_sizeof() == element.line_sizeof_" at /__w/milvus-lite/milvus-lite/milvus_binary/milvus/internal/core/src/query/Plan.cpp:48
 => vector dimension mismatch
attempt #2:context canceled
)>

In [33]:
#Stop the instance!
default_server.stop()
default_server.cleanup()